# Population Data Preprocessing

In [1]:
# Import dependencies
import pandas as pd

## Import Population Estimates

In [18]:
population_file_path = './data/PopulationEstimates.xlsx'
raw_population_df = pd.read_excel(population_file_path)
raw_population_df.head(5)

,"Decennial census population counts for the U.S., States, and counties, 1990-2020",Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9
0,FIPStxt,State,Area name,Rural-urban continuum code 2013,Population 1990,Population 2000,Population 2010,Population 2020,NaN,NaN
1,00000,US,United States,NaN,248790925,281424600,308745538,331449281,NaN,NaN
2,01000,AL,Alabama,NaN,4040389,4447207,4779736,5024279,NaN,NaN
3,01001,AL,Autauga County,2,34222,43751,54571,58805,NaN,NaN
4,01003,AL,Baldwin County,3,98280,140416,182265,231767,NaN,NaN


In [19]:
clean_population_df = raw_population_df.drop(columns=raw_population_df.columns[-2:])
clean_population_df.columns = clean_population_df.iloc[0]
clean_population_df = clean_population_df.drop(clean_population_df.index[0], axis=0)
clean_population_df.head()

,FIPStxt,State,Area name,Rural-urban continuum code 2013,Population 1990,Population 2000,Population 2010,Population 2020
1,00000,US,United States,NaN,248790925,281424600,308745538,331449281
2,01000,AL,Alabama,NaN,4040389,4447207,4779736,5024279
3,01001,AL,Autauga County,2,34222,43751,54571,58805
4,01003,AL,Baldwin County,3,98280,140416,182265,231767
5,01005,AL,Barbour County,6,25417,29042,27457,25223


In [20]:
pop_df = clean_population_df[['State', 'Population 2020']]
pop_df = pop_df.dropna(how='any')

pop_df = pop_df.rename(columns={'State': 'State Abrv'})
pop_df['Population 2020'] = pop_df['Population 2020'].apply(pd.to_numeric)
pop_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3274 entries, 1 to 3280
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   State Abrv       3274 non-null   object
 1   Population 2020  3274 non-null   int64 
dtypes: int64(1), object(1)
memory usage: 76.7+ KB


## Import State Acronyms

In [21]:
states_file_path = './data/us_state_mappings.csv'
raw_acronyms_df = pd.read_csv(states_file_path)
raw_acronyms_df.head(5)

,State,Abbrev,Code
0,Alabama,Ala.,AL
1,Alaska,Alaska,AK
2,Arizona,Ariz.,AZ
3,Arkansas,Ark.,AR
4,California,Calif.,CA


In [22]:
states_df = raw_acronyms_df.drop(columns=['Abbrev'])
states_df.rename(columns={'Code': 'State Abrv'}, inplace=True)
states_df = states_df.astype(str)
states_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   State       51 non-null     object
 1   State Abrv  51 non-null     object
dtypes: object(2)
memory usage: 944.0+ bytes


## Apply Mappings

In [23]:
merged_df = pop_df.merge(states_df, on='State Abrv')
merged_df.head()

,State Abrv,Population 2020,State
0,AL,5024279,Alabama
1,AL,58805,Alabama
2,AL,231767,Alabama
3,AL,25223,Alabama
4,AL,22293,Alabama


In [24]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3194 entries, 0 to 3193
Data columns (total 3 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   State Abrv       3194 non-null   object
 1   Population 2020  3194 non-null   int64 
 2   State            3194 non-null   object
dtypes: int64(1), object(2)
memory usage: 99.8+ KB


## Aggregate

In [27]:
state_populations = merged_df.groupby('State').sum()
state_populations.head()

,Population 2020
State,
Alabama,10048558
Alaska,1466782
Arizona,14303004
Arkansas,6023048
California,79076446


## Export

In [28]:
output_path = './data/state_population_data.csv'
state_populations.to_csv(output_path)